In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('../data/zomato_cleaned.csv')
df

,url,online_order,book_table,rate,votes,rest_type,approx_cost(for two people),listed_in(type),listed_in(city),Afghan,...,Spanish,Steak,Street Food,Sushi,Tamil,Tea,Thai,Tibetan,Turkish,Vietnamese
0,https://www.zomato.com/bangalore/jalsa-banasha...,1,1,4.1,775,Casual Dining,800,Buffet,Banashankari,0,...,0,0,0,0,0,0,0,0,0,0
1,https://www.zomato.com/bangalore/spice-elephan...,1,0,4.1,787,Casual Dining,800,Buffet,Banashankari,0,...,0,0,0,0,0,0,0,0,0,0
2,https://www.zomato.com/SanchurroBangalore?cont...,1,0,3.8,918,"Cafe, Casual Dining",800,Buffet,Banashankari,0,...,0,0,0,0,0,0,0,0,0,0
3,https://www.zomato.com/bangalore/addhuri-udupi...,0,0,3.7,88,Quick Bites,300,Buffet,Banashankari,0,...,0,0,0,0,0,0,0,0,0,0
4,https://www.zomato.com/bangalore/grand-village...,0,0,3.8,166,Casual Dining,600,Buffet,Banashankari,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9032,https://www.zomato.com/bangalore/chung-wah-itp...,0,0,2.5,73,Casual Dining,800,Dine-out,Whitefield,0,...,0,0,0,0,0,0,0,0,0,0
9033,https://www.zomato.com/bangalore/nawabs-empire...,0,0,3.2,5,Quick Bites,300,Dine-out,Whitefield,0,...,0,0,0,0,0,0,0,0,0,0
9034,https://www.zomato.com/bangalore/fujian-expres...,1,0,3.8,182,Food Court,600,Dine-out,Whitefield,0,...,0,0,0,0,0,0,1,0,0,0
9035,https://www.zomato.com/bangalore/seeya-restaur...,0,0,3.3,4,Quick Bites,350,Dine-out,Whitefield,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
x = df.drop(['rate', 'url'], axis=1)
y = df['rate']

In [4]:
cat = x.select_dtypes(include='object').columns
numeric = x.select_dtypes(exclude='object').columns

In [5]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(drop='first')
cat_data = ohe.fit_transform(x[cat]).toarray() 

In [6]:
x = pd.concat([x[numeric], pd.DataFrame(cat_data, columns=ohe.get_feature_names(cat))], axis=1)

c:\Users\Yousef\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [7]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x , y , test_size=0.2, random_state= 42)

In [8]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [9]:
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, classification_report, plot_confusion_matrix

In [10]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(x_train_scaled,y_train)
y_pred_lr = lr.predict(x_test_scaled)
print('R2 for training',lr.score(x_train_scaled,y_train))
print('R2 for testing',lr.score(x_test_scaled,y_test))
print('MAPE for training',mean_absolute_percentage_error(y_train, lr.predict(x_train_scaled)))
print('MAPE for testing',mean_absolute_percentage_error(y_test, y_pred_lr))
print('MSE for training',mean_squared_error(y_train, lr.predict(x_train_scaled)))
print('MSE for testing',mean_squared_error(y_test, y_pred_lr))
dt_dataframe = pd.DataFrame({'Actual': y_test, 'Predicted': np.round(y_pred_lr, 1)})
dt_dataframe

R2 for training 0.30944235496935013
R2 for testing -5.342671978217058e+23
MAPE for training 0.08068875974987255
MAPE for testing 4835221532.704403
MSE for training 0.12848800136126912
MSE for testing 9.788753508368205e+22


,Actual,Predicted
1922,3.5,3.5
1318,4.2,4.7
4787,3.3,3.6
2115,3.2,3.2
5689,3.2,3.3
...,...,...
6068,3.9,3.6
3245,4.0,3.6
5327,4.1,4.0
1111,3.1,3.5


In [11]:
from sklearn.tree import DecisionTreeRegressor
dt = DecisionTreeRegressor()
dt.fit(x_train_scaled,y_train)
y_pred_dtr = dt.predict(x_test_scaled)
print('R2 for training',dt.score(x_train_scaled,y_train))
print('R2 for testing',dt.score(x_test_scaled,y_test))
print('MAPE for training',mean_absolute_percentage_error(y_train, dt.predict(x_train_scaled)))
print('MAPE for testing',mean_absolute_percentage_error(y_test, y_pred_dtr))
print('MSE for training',mean_squared_error(y_train, dt.predict(x_train_scaled)))
print('MSE for testing',mean_squared_error(y_test, y_pred_dtr))
dt_dataframe = pd.DataFrame({'Actual': y_test, 'Predicted': np.round(y_pred_dtr, 1)})
dt_dataframe

R2 for training 0.9993606221448256
R2 for testing -0.061383408267356554
MAPE for training 0.00010437251540657695
MAPE for testing 0.0884478087916269
MSE for training 0.00011896527873841475
MSE for testing 0.19446487831858406


,Actual,Predicted
1922,3.5,3.3
1318,4.2,4.4
4787,3.3,3.3
2115,3.2,3.0
5689,3.2,2.9
...,...,...
6068,3.9,4.1
3245,4.0,3.9
5327,4.1,3.6
1111,3.1,3.8


In [12]:
dt_dataframe[dt_dataframe['Actual'] != dt_dataframe['Predicted']]

,Actual,Predicted
1922,3.5,3.3
1318,4.2,4.4
2115,3.2,3.0
5689,3.2,2.9
8149,3.1,3.6
...,...,...
6068,3.9,4.1
3245,4.0,3.9
5327,4.1,3.6
1111,3.1,3.8


In [13]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(x_train_scaled,y_train)
y_pred_rf = rf.predict(x_test_scaled)
print('R2 for training',rf.score(x_train_scaled,y_train))
print('R2 for testing',rf.score(x_test_scaled,y_test))
print('MAPE for training',mean_absolute_percentage_error(y_train, rf.predict(x_train_scaled)))
print('MAPE for testing',mean_absolute_percentage_error(y_test, y_pred_rf))
print('MSE for training',mean_squared_error(y_train, rf.predict(x_train_scaled)))
print('MSE for testing',mean_squared_error(y_test, y_pred_rf))
rf_dataframe = pd.DataFrame({'Actual': y_test, 'Predicted': np.round(y_pred_rf, 1)})
rf_dataframe

R2 for training 0.921773042740035
R2 for testing 0.42316043994164554
MAPE for training 0.025212272079175677
MAPE for testing 0.06977209229339087
MSE for training 0.014555230056804615
MSE for testing 0.10568757150558095


,Actual,Predicted
1922,3.5,3.4
1318,4.2,4.4
4787,3.3,3.3
2115,3.2,3.2
5689,3.2,3.2
...,...,...
6068,3.9,3.9
3245,4.0,3.8
5327,4.1,4.0
1111,3.1,3.5


In [14]:
from sklearn.svm import SVR
svm = SVR()
svm.fit(x_train_scaled,y_train)
y_predsvmf = svm.predict(x_test_scaled)
print('R2 for training',svm.score(x_train_scaled,y_train))
print('R2 for testing',svm.score(x_test_scaled,y_test))
print('MAPE for training',mean_absolute_percentage_error(y_train, svm.predict(x_train_scaled)))
print('MAPE for testing',mean_absolute_percentage_error(y_test, y_predsvmf))
print('MSE for training',mean_squared_error(y_train, svm.predict(x_train_scaled)))
print('MSE for testing',mean_squared_error(y_test, y_predsvmf))
svm_dataframe = pd.DataFrame({'Actual': y_test, 'Predicted': np.round(y_predsvmf, 1)})
svm_dataframe


R2 for training 0.5471402464356179
R2 for testing 0.31460560686701133
MAPE for training 0.06217611090279268
MAPE for testing 0.08097142711059331
MSE for training 0.0842609520742642
MSE for testing 0.12557680497232027


,Actual,Predicted
1922,3.5,3.5
1318,4.2,4.1
4787,3.3,3.4
2115,3.2,3.2
5689,3.2,3.3
...,...,...
6068,3.9,3.8
3245,4.0,3.9
5327,4.1,3.9
1111,3.1,3.9


In [15]:
svm_dataframe[svm_dataframe['Actual'] != svm_dataframe['Predicted']]

,Actual,Predicted
1318,4.2,4.1
4787,3.3,3.4
5689,3.2,3.3
8149,3.1,3.5
8310,3.0,3.5
...,...,...
6068,3.9,3.8
3245,4.0,3.9
5327,4.1,3.9
1111,3.1,3.9


In [16]:
from  xgboost import XGBRegressor
xgb = XGBRegressor()
xgb.fit(x_train_scaled,y_train)
y_pred_xgb = xgb.predict(x_test_scaled)
print('R2 for training',svm.score(x_train_scaled,y_train))
print('R2 for testing',svm.score(x_test_scaled,y_test))
print('MAPE for training',mean_absolute_percentage_error(y_train, xgb.predict(x_train_scaled)))
print('MAPE for testing',mean_absolute_percentage_error(y_test, y_pred_xgb))
print('MSE for training',mean_squared_error(y_train, xgb.predict(x_train_scaled)))
print('MSE for testing',mean_squared_error(y_test, y_pred_xgb))

xgb_dataframe = pd.DataFrame({'Actual': y_test, 'Predicted': np.round(y_pred_xgb, 1)})
xgb_dataframe



R2 for training 0.5471402464356179
R2 for testing 0.31460560686701133
MAPE for training 0.05174123269054359
MAPE for testing 0.0700205303526976
MSE for training 0.05874609927369819
MSE for testing 0.10380739409276846


,Actual,Predicted
1922,3.5,3.2
1318,4.2,4.5
4787,3.3,3.3
2115,3.2,3.2
5689,3.2,3.2
...,...,...
6068,3.9,3.8
3245,4.0,3.6
5327,4.1,3.9
1111,3.1,3.1


In [20]:
y_test_calssified = y_test.apply(lambda x: 1 if x > 3.75 else 0)
y_pred_calssified = pd.Series(y_pred_dtr).apply(lambda x: 1 if x > 3.75 else 0)

In [21]:
print(classification_report(y_test_calssified, y_pred_calssified))

              precision    recall  f1-score   support

           0       0.78      0.78      0.78      1060
           1       0.69      0.69      0.69       748

    accuracy                           0.74      1808
   macro avg       0.74      0.74      0.74      1808
weighted avg       0.74      0.74      0.74      1808

